In [35]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv2DTranspose
import os
import cv2
import numpy as np

In [36]:
conv_kwargs = {
    "padding"             : "SAME",
    "activation"          : keras.layers.LeakyReLU(alpha=0.2),
    "kernel_initializer"  : tf.random_normal_initializer(stddev=.1)
}

In [33]:
class Autoencoder(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.encoder = keras.Sequential([
             Conv2D(16, 8, 2, **conv_kwargs),
            Conv2D(16, 8, 2, **conv_kwargs),
            keras.layers.BatchNormalization(),
            Conv2D(32, 4, 1, **conv_kwargs),
            Conv2D(32, 4, 2, **conv_kwargs),
            keras.layers.BatchNormalization(),
            Conv2D(64, 4, 1, **conv_kwargs),
            Conv2D(64, 2, 1, padding='same', 
                   kernel_initializer=tf.random_normal_initializer(stddev=.1))
        ], name="ae_encoder")

        self.decoder = keras.Sequential([
        Conv2DTranspose(64, 4, 1, **conv_kwargs),
        Conv2DTranspose(16, 8, 2, **conv_kwargs),
        Conv2DTranspose(3, 8, 4, padding='same', kernel_initializer=tf.random_normal_initializer(stddev=.1))
        
    ], name='ae_decoder')

    def call(self, inputs):
        inputs = self.encoder(inputs)
        inputs = self.decoder(inputs)
        return inputs


In [37]:
def custom_loss(y_true, y_pred):
    mse_loss = keras.losses.MeanSquaredError()
    bce_loss = keras.losses.BinaryCrossentropy()
    mae_loss = keras.losses.MeanAbsoluteError()
    mse = mse_loss(y_true, y_pred)
    #bce = bce_loss(y_true, y_pred)
    mae = mae_loss(y_true, y_pred)
    loss = .3*mse + .7*mae
    return loss

In [34]:
ae_model = Autoencoder(name='autoencoder')

ae_model.build(input_shape = (1,256,256,3))   ## Required to see architecture summary
initial_weights = ae_model.get_weights() ## Just so we can reset out autoencoder

ae_model.summary()
ae_model.encoder.summary()
ae_model.decoder.summary()

ae_model.compile(
    optimizer   = keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss        = custom_loss,
    metrics     = [
        tf.keras.metrics.MeanSquaredError(),
        #tf.keras.metrics.BinaryCrossentropy()
        tf.keras.metrics.MeanAbsoluteError()
    ]
)

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ae_encoder (Sequential)     (1, 32, 32, 64)           93600     
                                                                 
 ae_decoder (Sequential)     (1, 256, 256, 3)          134227    
                                                                 
Total params: 227827 (889.95 KB)
Trainable params: 227731 (889.57 KB)
Non-trainable params: 96 (384.00 Byte)
_________________________________________________________________
Model: "ae_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (1, 128, 128, 16)         3088      
                                                                 
 conv2d_28 (Conv2D)          (1, 64, 64, 16)           16400     
                                                          

In [9]:
import sys

isColab = "google.colab" in sys.modules
data_dir = '../collapsed_data'
# this also works:
# isColab = "COLAB_GPU" in os.environ

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    data_dir = ("/content/drive/MyDrive/collapsed_data")

In [10]:
def get_data(sub_dir, size=(256,256)):
    dir = os.path.join(data_dir, sub_dir)
    files = os.listdir(dir)
    x = []
    y = []
    for f in files:
        path = os.path.join(dir, f)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, size)
            y.append(img)
        except:
            print(path)
            continue
    y = np.array(y, dtype=np.float32)
    x = np.copy(y)
    h,w = size
    rec_w = w//5
    # Make middle black
    x[:,:,rec_w*2:rec_w*3,:]=0
    y=y/255
    x = x/255
    print(x.shape)
    print(y.shape)
    print('done')
    return x,y


In [26]:
x_train,y_train = get_data('train')
x_valid,y_valid = get_data('validation')

../collapsed_data/train/Glacier-Train (558).jpeg


In [ ]:
# Train the model
print('Fitting model')
ae_model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_valid,y_valid))
#ae_model.fit(x_train, y_train, epochs=5, batch_size=64)
print('------------------------')

In [ ]:
ae_model.save('model.keras')

In [16]:
ae_model = keras.models.load_model('../models/MseMaeWeighted.keras',custom_objects={'Autoencoder': Autoencoder,'custom_loss':custom_loss})

/Users/pratham/anaconda3/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tensorflow.python.ops.init_ops_v2.RandomNormal'>, which may lead to improper serialization.
  warnings.warn(


In [17]:
print('Evaluating model on testing data')
x_test,y_test = get_data('test')
ae_model.evaluate(x_test, y_test, batch_size=32)

Evaluating model on testing data
(500, 256, 256, 3)
(500, 256, 256, 3)
done
16/16 [==============================] - 1s 43ms/step - loss: 0.0524 - mean_squared_error: 0.0106 - mean_absolute_error: 0.0703


[0.052372947335243225, 0.010550432838499546, 0.07029688358306885]

In [25]:
x = x_test[:5]
pred = ae_model.predict(x)
for i in range(5):
    cv2.imshow('im', x[i])
    cv2.waitKeyEx()
    cv2.imshow('pred', pred[i])
    cv2.waitKeyEx()

cv2.destroyAllWindows()
cv2.waitKey(1)

1/1 [==============================] - 0s 23ms/step


-1